

Input:
- tables_nov/

Output:
- cpuep_results/aggregated/
- cudaep_results/aggregated/

In [240]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import itertools  # for cycling through colors if necessary
import glob
import re
import os

In [241]:
"""
grouped_by_size_range = df.groupby(['size_group','config'])[numeric_columns].mean().reset_index()

"""

"\ngrouped_by_size_range = df.groupby(['size_group','config'])[numeric_columns].mean().reset_index()\n\n"

In [344]:
num_models = 12 #CHANGE
ep = 'cudaep' #change
#path = f"D:/GAISSA/energy-repo/last_repo/june_{device}/"  # Adjust this path to your CSV files location #CHANGE

path = None
if ep == 'cpuep':
    path = f"D:/GAISSA/energy-repo/repo_sept_last_update/cpuep_nov_results/nov_cpuep_00/" #
else:
    path = f"D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/nov_cudaep_00/" # 

#save_figs = False
SAVE_FIGS = True
SAVE_TABLES = True
UPDATE_TINY = False

In [345]:
if ep == 'cpuep':
    path_to_save = f"D:/GAISSA/energy-repo/repo_sept_last_update/cpuep_nov_results/"
else:
    path_to_save = f"D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/" # change


In [346]:
path_to_save

'D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/'

In [347]:
save_dir = path_to_save+"aggregated/"
save_dir

'D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/aggregated/'

In [348]:


try:
    os.mkdir(save_dir)
except Exception as e:
    print("could not create directory "+ save_dir )
    print(e)



could not create directory D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/aggregated/
[WinError 183] Cannot create a file when that file already exists: 'D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/aggregated/'


In [349]:
#save_dir = path+"figures_nov/"
tables_dir = path+"tables_nov/"
print(tables_dir)

D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/nov_cudaep_00/tables_nov/


In [350]:
def remove_condition_rows(condition_to_remove,df):
    # example: condition = merged_df['label']  == 'idle_idle'
    df1 = df.copy()
    return df1[~condition_to_remove]

In [351]:
def save_latex_table(df,save_dir):
    df = df.copy()
    df.columns = [col.replace('_', ' ') for col in df.columns]
    
    latex_table = df.to_latex(index=False)

    # Define the filename
    filename = save_dir
    
    # Open the file in write mode
    with open(filename, 'w') as file:
        file.write(latex_table) if SAVE_TABLES else print(f"SAVE_TABLES:{SAVE_TABLES}")
        
    # Print LaTeX table
    print(latex_table)

In [250]:
# Updated model and size data
model_data = {
    'Model': [
        'codeparrot-small', 'tiny-starcoder', 'pythia-410m', 'bloomz-560m',
        'starcoderbase-1b', 'bloomz-1b1', 'tinyllama', 'pythia1-4b',
        'codegemma-2b', 'phi2', 'stablecode-3b', 'stablecode-3b-completion'
    ],
    'Size': [110, 164, 410, 560, 1000, 1100, 1100, 1400, 2000, 2700, 3000, 3000]  # Model sizes in millions
}

# Create a new DataFrame for models
model_df = pd.DataFrame(model_data)

# Define size ranges based on provided groups
def assign_size_range(size):
    if size < 1000:
        return 1
    elif 1000 <= size < 2000:
        return 2
    else:
        return 3

model_df['size_group'] = model_df['Size'].apply(assign_size_range)
model_df

,Model,Size,size_group
0,codeparrot-small,110,1
1,tiny-starcoder,164,1
2,pythia-410m,410,1
3,bloomz-560m,560,1
4,starcoderbase-1b,1000,2
5,bloomz-1b1,1100,2
6,tinyllama,1100,2
7,pythia1-4b,1400,2
8,codegemma-2b,2000,3
9,phi2,2700,3


## Energibridge x

In [17]:
tables_dir

'D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/nov_cudaep_00/tables_nov/'

In [18]:
energi_bridge_ag_01 = pd.read_csv(tables_dir + 'final_energibridge_data.csv',index_col=None)
energi_bridge_ag_01

,label,experiment,avg_cpu_usage_config,avg_USED_MEMORY,avg_used_memory_pct_config
0,idle_idle,1,0.143838,2.794639e+09,4.197672
1,idle_idle,1,0.143838,2.794639e+09,4.197672
2,idle_idle,1,0.143838,2.794639e+09,4.197672
3,idle_idle,1,0.143838,2.794639e+09,4.197672
4,idle_idle,1,0.143838,2.794639e+09,4.197672
...,...,...,...,...,...
542672,torchscript_stablecode-3b-completion,9,0.598001,3.733427e+09,5.607227
542673,torchscript_stablecode-3b-completion,9,0.598001,3.733427e+09,5.607227
542674,torchscript_stablecode-3b-completion,9,0.598001,3.733427e+09,5.607227
542675,torchscript_stablecode-3b-completion,9,0.598001,3.733427e+09,5.607227


In [19]:
energi_bridge_ag_01.columns

Index(['label', 'experiment', 'avg_cpu_usage_config', 'avg_USED_MEMORY',
       'avg_used_memory_pct_config'],
      dtype='object')

In [20]:
energi_variables = ['avg_cpu_usage_config', 'avg_USED_MEMORY','avg_used_memory_pct_config']
energi_bridge_ag_02 = energi_bridge_ag_01.groupby(['label','experiment'])[energi_variables].mean().reset_index()
energi_bridge_ag_02

,label,experiment,avg_cpu_usage_config,avg_USED_MEMORY,avg_used_memory_pct_config
0,idle_idle,1,0.143838,2.794639e+09,4.197672
1,idle_idle,2,0.114619,2.793104e+09,4.195367
2,idle_idle,3,0.088746,2.131002e+09,3.200859
3,idle_idle,4,0.091595,2.135717e+09,3.207942
4,idle_idle,5,0.103151,2.146677e+09,3.224404
...,...,...,...,...,...
365,torchscript_tinyllama,6,0.519750,7.166762e+09,10.764794
366,torchscript_tinyllama,7,0.535989,7.167283e+09,10.765576
367,torchscript_tinyllama,8,0.554561,9.198180e+09,13.816070
368,torchscript_tinyllama,9,0.506095,9.067365e+09,13.619581


In [21]:
energi_bridge_ag_03 = energi_bridge_ag_02.groupby(['label'])[energi_variables].median().reset_index()
energi_bridge_ag_03

,label,avg_cpu_usage_config,avg_USED_MEMORY,avg_used_memory_pct_config
0,idle_idle,0.110316,2.495251e+09,3.747977
1,onnx_bloomz-1b1,3.205927,6.276978e+09,9.427382
2,onnx_bloomz-560m,3.244376,3.267366e+09,4.907251
3,onnx_codegemma-2b,3.211293,3.873413e+09,5.817473
4,onnx_codeparrot-small,3.995175,5.235244e+09,7.863568
5,onnx_phi2,3.422455,6.020537e+09,9.043114
6,onnx_pythia-410m,3.426432,6.674936e+09,10.026048
7,onnx_pythia1-4b,3.381862,4.722682e+09,7.093677
8,onnx_stablecode-3b,3.278702,3.553119e+09,5.336423
9,onnx_stablecode-3b-completion,2.974873,3.791030e+09,5.693742


In [22]:
# Split the 'label' column into 'runtime' and 'model'
energi_bridge_ag_03[['runtime', 'model']] = energi_bridge_ag_03['label'].str.split('_', n=1, expand=True)
energi_bridge_ag_03

,label,avg_cpu_usage_config,avg_USED_MEMORY,avg_used_memory_pct_config,runtime,model
0,idle_idle,0.110316,2.495251e+09,3.747977,idle,idle
1,onnx_bloomz-1b1,3.205927,6.276978e+09,9.427382,onnx,bloomz-1b1
2,onnx_bloomz-560m,3.244376,3.267366e+09,4.907251,onnx,bloomz-560m
3,onnx_codegemma-2b,3.211293,3.873413e+09,5.817473,onnx,codegemma-2b
4,onnx_codeparrot-small,3.995175,5.235244e+09,7.863568,onnx,codeparrot-small
5,onnx_phi2,3.422455,6.020537e+09,9.043114,onnx,phi2
6,onnx_pythia-410m,3.426432,6.674936e+09,10.026048,onnx,pythia-410m
7,onnx_pythia1-4b,3.381862,4.722682e+09,7.093677,onnx,pythia1-4b
8,onnx_stablecode-3b,3.278702,3.553119e+09,5.336423,onnx,stablecode-3b
9,onnx_stablecode-3b-completion,2.974873,3.791030e+09,5.693742,onnx,stablecode-3b-completion


,Model,Size,size_group
0,codeparrot-small,110,1
1,tiny-starcoder,164,1
2,pythia-410m,410,1
3,bloomz-560m,560,1
4,starcoderbase-1b,1000,2
5,bloomz-1b1,1100,2
6,tinyllama,1100,2
7,pythia1-4b,1400,2
8,codegemma-2b,2000,3
9,phi2,2700,3


In [24]:
# Map the Size Range from model_df to df_to_save
energi_bridge_ag_03_range = energi_bridge_ag_03.merge(model_df[['Model', 'size_group']], left_on='model', right_on='Model', how='left')

# Drop the redundant 'Model' column after merging
energi_bridge_ag_03_range = energi_bridge_ag_03_range.drop(columns=['Model'])
energi_bridge_ag_03_range

,label,avg_cpu_usage_config,avg_USED_MEMORY,avg_used_memory_pct_config,runtime,model,size_group
0,idle_idle,0.110316,2.495251e+09,3.747977,idle,idle,NaN
1,onnx_bloomz-1b1,3.205927,6.276978e+09,9.427382,onnx,bloomz-1b1,2.0
2,onnx_bloomz-560m,3.244376,3.267366e+09,4.907251,onnx,bloomz-560m,1.0
3,onnx_codegemma-2b,3.211293,3.873413e+09,5.817473,onnx,codegemma-2b,3.0
4,onnx_codeparrot-small,3.995175,5.235244e+09,7.863568,onnx,codeparrot-small,1.0
5,onnx_phi2,3.422455,6.020537e+09,9.043114,onnx,phi2,3.0
6,onnx_pythia-410m,3.426432,6.674936e+09,10.026048,onnx,pythia-410m,1.0
7,onnx_pythia1-4b,3.381862,4.722682e+09,7.093677,onnx,pythia1-4b,2.0
8,onnx_stablecode-3b,3.278702,3.553119e+09,5.336423,onnx,stablecode-3b,3.0
9,onnx_stablecode-3b-completion,2.974873,3.791030e+09,5.693742,onnx,stablecode-3b-completion,3.0


In [25]:
# Selecting only numeric columns for grouping

energi_bridge_ag_04 =energi_bridge_ag_03_range.groupby(['size_group','runtime'])[energi_variables].mean().reset_index()
energi_bridge_ag_04

,size_group,runtime,avg_cpu_usage_config,avg_USED_MEMORY,avg_used_memory_pct_config
0,1.0,onnx,3.670914,4.990815e+09,7.496130
1,1.0,torch,2.080496,3.612457e+09,5.425793
2,1.0,torchscript,0.208496,6.971902e+09,10.471826
3,2.0,onnx,3.472886,4.962850e+09,7.454043
4,2.0,torch,2.185318,3.380719e+09,5.077734
5,2.0,torchscript,0.382377,5.269584e+09,7.914890
6,3.0,onnx,3.221830,4.309525e+09,6.472688
7,3.0,torch,2.642276,4.777427e+09,7.175488
8,3.0,torchscript,0.615369,5.197203e+09,7.806012


In [26]:
save_dir

'D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/aggregated/'

In [27]:
energi_datasets = [energi_bridge_ag_01, energi_bridge_ag_02, energi_bridge_ag_03, energi_bridge_ag_04,]
SAVE_TABLES = True # change
agg_level = 1 # change
for dataset in energi_datasets:
    df_name = f"energibridge_vars_{agg_level}.csv"
    dataset.to_csv(save_dir+df_name, index=False)  if SAVE_TABLES else print(f"SAVE_TABLES:{SAVE_TABLES}")   
    print(f"Saving in: {save_dir+df_name}")
    agg_level+=1

Saving in: D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/aggregated/energibridge_vars_1.csv
Saving in: D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/aggregated/energibridge_vars_2.csv
Saving in: D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/aggregated/energibridge_vars_3.csv
Saving in: D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/aggregated/energibridge_vars_4.csv


## Nvidia vars x

In [28]:
nvidia_ag_01 = pd.read_csv(tables_dir + 'final_nvidia_data.csv',index_col=None)
nvidia_ag_01

,label,experiment,config_duration,avg_utilization_gpu_config,avg_utilization_memory_config,avg_memory_used_config,avg_power_draw_config,avg_temperature_gpu_config,avg_used_memory_pct_mib
0,torch_pythia1-4b,1,193.242,73.584680,67.165711,6061.018760,178.655961,49.400730,24.674397
1,torch_pythia1-4b,1,193.242,73.584680,67.165711,6061.018760,178.655961,49.400730,24.674397
2,torch_pythia1-4b,1,193.242,73.584680,67.165711,6061.018760,178.655961,49.400730,24.674397
3,torch_pythia1-4b,1,193.242,73.584680,67.165711,6061.018760,178.655961,49.400730,24.674397
4,torch_pythia1-4b,1,193.242,73.584680,67.165711,6061.018760,178.655961,49.400730,24.674397
...,...,...,...,...,...,...,...,...,...
1047986,torchscript_stablecode-3b-completion,9,107.748,5.041199,2.423221,20944.451311,69.656236,36.781835,85.264824
1047987,torchscript_stablecode-3b-completion,9,107.748,5.041199,2.423221,20944.451311,69.656236,36.781835,85.264824
1047988,torchscript_stablecode-3b-completion,9,107.748,5.041199,2.423221,20944.451311,69.656236,36.781835,85.264824
1047989,torchscript_stablecode-3b-completion,9,107.748,5.041199,2.423221,20944.451311,69.656236,36.781835,85.264824


In [29]:
nvidia_ag_01.columns

Index(['label', 'experiment', 'config_duration', 'avg_utilization_gpu_config',
       'avg_utilization_memory_config', 'avg_memory_used_config',
       'avg_power_draw_config', 'avg_temperature_gpu_config',
       'avg_used_memory_pct_mib'],
      dtype='object')

In [31]:
nvidia_variables = ['config_duration', 'avg_utilization_gpu_config',
       'avg_utilization_memory_config', 'avg_memory_used_config',
       'avg_used_memory_pct_mib']
nvidia_ag_02 = nvidia_ag_01.groupby(['label','experiment'])[nvidia_variables].mean().reset_index()
nvidia_ag_02

,label,experiment,config_duration,avg_utilization_gpu_config,avg_utilization_memory_config,avg_memory_used_config,avg_used_memory_pct_mib
0,onnx_bloomz-1b1,1,1046.176,17.931602,2.343146,2648.308129,10.781258
1,onnx_bloomz-1b1,2,1039.802,18.035811,2.330817,2625.757259,10.689453
2,onnx_bloomz-1b1,3,1157.241,16.228949,2.094990,3134.658490,12.761189
3,onnx_bloomz-1b1,4,1039.818,18.074712,2.333785,2712.607278,11.043019
4,onnx_bloomz-1b1,5,1038.632,18.064722,2.341730,2629.047573,10.702848
...,...,...,...,...,...,...,...
355,torchscript_tinyllama,6,107.464,2.165103,0.922139,13469.904315,54.835956
356,torchscript_tinyllama,7,107.188,2.160865,0.920978,13449.905927,54.754543
357,torchscript_tinyllama,8,107.658,2.134831,0.904494,13469.906367,54.835965
358,torchscript_tinyllama,9,107.137,2.174036,0.922860,13602.919097,55.377459


In [32]:
nvidia_ag_03 = nvidia_ag_02.groupby(['label'])[nvidia_variables].median().reset_index()
nvidia_ag_03

,label,config_duration,avg_utilization_gpu_config,avg_utilization_memory_config,avg_memory_used_config,avg_used_memory_pct_mib
0,onnx_bloomz-1b1,1039.7475,18.066792,2.337708,2671.332011,10.874988
1,onnx_bloomz-560m,230.1520,51.547404,8.677683,3378.865261,13.755354
2,onnx_codegemma-2b,694.6140,75.492872,27.041037,9259.542459,37.695581
3,onnx_codeparrot-small,193.1545,23.249946,5.922577,14058.657538,57.232770
4,onnx_phi2,807.0430,69.617695,27.326302,8632.553030,35.143108
5,onnx_pythia-410m,452.2400,31.098195,7.978069,14532.227572,59.160672
6,onnx_pythia1-4b,543.4775,63.381850,24.519520,17653.391739,71.866926
7,onnx_stablecode-3b,631.1805,65.564792,14.174740,7576.162447,30.842544
8,onnx_stablecode-3b-completion,978.5415,64.457540,27.197272,9500.931137,38.678274
9,onnx_starcoderbase-1b,564.1585,49.946659,20.115380,3229.053382,13.145471


In [33]:
# Split the 'label' column into 'runtime' and 'model'
nvidia_ag_03[['runtime', 'model']] = nvidia_ag_03['label'].str.split('_', n=1, expand=True)
nvidia_ag_03

,label,config_duration,avg_utilization_gpu_config,avg_utilization_memory_config,avg_memory_used_config,avg_used_memory_pct_mib,runtime,model
0,onnx_bloomz-1b1,1039.7475,18.066792,2.337708,2671.332011,10.874988,onnx,bloomz-1b1
1,onnx_bloomz-560m,230.1520,51.547404,8.677683,3378.865261,13.755354,onnx,bloomz-560m
2,onnx_codegemma-2b,694.6140,75.492872,27.041037,9259.542459,37.695581,onnx,codegemma-2b
3,onnx_codeparrot-small,193.1545,23.249946,5.922577,14058.657538,57.232770,onnx,codeparrot-small
4,onnx_phi2,807.0430,69.617695,27.326302,8632.553030,35.143108,onnx,phi2
5,onnx_pythia-410m,452.2400,31.098195,7.978069,14532.227572,59.160672,onnx,pythia-410m
6,onnx_pythia1-4b,543.4775,63.381850,24.519520,17653.391739,71.866926,onnx,pythia1-4b
7,onnx_stablecode-3b,631.1805,65.564792,14.174740,7576.162447,30.842544,onnx,stablecode-3b
8,onnx_stablecode-3b-completion,978.5415,64.457540,27.197272,9500.931137,38.678274,onnx,stablecode-3b-completion
9,onnx_starcoderbase-1b,564.1585,49.946659,20.115380,3229.053382,13.145471,onnx,starcoderbase-1b


In [34]:
# Map the Size Range from model_df to df_to_save
nvidia_ag_03_range = nvidia_ag_03.merge(model_df[['Model', 'size_group']], left_on='model', right_on='Model', how='left')

# Drop the redundant 'Model' column after merging
nvidia_ag_03_range = nvidia_ag_03_range.drop(columns=['Model'])
nvidia_ag_03_range

,label,config_duration,avg_utilization_gpu_config,avg_utilization_memory_config,avg_memory_used_config,avg_used_memory_pct_mib,runtime,model,size_group
0,onnx_bloomz-1b1,1039.7475,18.066792,2.337708,2671.332011,10.874988,onnx,bloomz-1b1,2
1,onnx_bloomz-560m,230.1520,51.547404,8.677683,3378.865261,13.755354,onnx,bloomz-560m,1
2,onnx_codegemma-2b,694.6140,75.492872,27.041037,9259.542459,37.695581,onnx,codegemma-2b,3
3,onnx_codeparrot-small,193.1545,23.249946,5.922577,14058.657538,57.232770,onnx,codeparrot-small,1
4,onnx_phi2,807.0430,69.617695,27.326302,8632.553030,35.143108,onnx,phi2,3
5,onnx_pythia-410m,452.2400,31.098195,7.978069,14532.227572,59.160672,onnx,pythia-410m,1
6,onnx_pythia1-4b,543.4775,63.381850,24.519520,17653.391739,71.866926,onnx,pythia1-4b,2
7,onnx_stablecode-3b,631.1805,65.564792,14.174740,7576.162447,30.842544,onnx,stablecode-3b,3
8,onnx_stablecode-3b-completion,978.5415,64.457540,27.197272,9500.931137,38.678274,onnx,stablecode-3b-completion,3
9,onnx_starcoderbase-1b,564.1585,49.946659,20.115380,3229.053382,13.145471,onnx,starcoderbase-1b,2


In [37]:
#nvidia_ag_03_range.sort_values(by='size_group')

In [38]:

nvidia_ag_04 =nvidia_ag_03_range.groupby(['size_group','runtime'])[nvidia_variables].mean().reset_index()
nvidia_ag_04

,size_group,runtime,config_duration,avg_utilization_gpu_config,avg_utilization_memory_config,avg_memory_used_config,avg_used_memory_pct_mib
0,1,onnx,280.638000,33.617375,7.219379,8307.111365,33.818235
1,1,torch,125.313375,29.927797,15.321538,9807.115161,39.924748
2,1,torchscript,97.594875,0.922533,0.247153,16278.381512,66.269262
3,2,onnx,638.725875,49.013003,18.058188,10095.224278,41.097640
4,2,torch,171.074250,56.092893,49.308426,5993.570712,24.399816
5,2,torchscript,102.218875,2.474348,1.044582,15927.253803,64.839822
6,3,onnx,777.844750,68.783225,23.934838,8742.297268,35.589877
7,3,torch,264.918500,73.681243,69.027321,12207.871861,49.698224
8,3,torchscript,108.591625,4.955172,2.330681,18061.714258,73.529206


In [39]:
save_dir

'D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/aggregated/'

In [40]:
nvidia_datasets = [nvidia_ag_01,nvidia_ag_02,nvidia_ag_03,nvidia_ag_04,]
SAVE_TABLES = True # change
agg_level = 1 # change
for dataset in nvidia_datasets:
    df_name = f"nvidia_vars_{agg_level}.csv"
    dataset.to_csv(save_dir+df_name, index=False)  if SAVE_TABLES else print(f"SAVE_TABLES:{SAVE_TABLES}")   
    print(f"Saving in: {save_dir+df_name}")
    agg_level+=1

Saving in: D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/aggregated/nvidia_vars_1.csv
Saving in: D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/aggregated/nvidia_vars_2.csv
Saving in: D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/aggregated/nvidia_vars_3.csv
Saving in: D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/aggregated/nvidia_vars_4.csv


##  execution time or Energy, energibridge and nvidia too

In [313]:
#df_ag_01 = pd.read_csv(tables_dir + 'final_energibridge_data.csv',index_col=None)
df_ag_01 = pd.read_csv(tables_dir + 'final_nvidia_data.csv',index_col=None)

df_ag_01

,label,experiment,config_duration,avg_utilization_gpu_config,avg_utilization_memory_config,avg_memory_used_config,avg_power_draw_config,avg_temperature_gpu_config,avg_used_memory_pct_mib
0,torch_pythia1-4b,1,193.242,73.584680,67.165711,6061.018760,178.655961,49.400730,24.674397
1,torch_pythia1-4b,1,193.242,73.584680,67.165711,6061.018760,178.655961,49.400730,24.674397
2,torch_pythia1-4b,1,193.242,73.584680,67.165711,6061.018760,178.655961,49.400730,24.674397
3,torch_pythia1-4b,1,193.242,73.584680,67.165711,6061.018760,178.655961,49.400730,24.674397
4,torch_pythia1-4b,1,193.242,73.584680,67.165711,6061.018760,178.655961,49.400730,24.674397
...,...,...,...,...,...,...,...,...,...
1047986,torchscript_stablecode-3b-completion,9,107.748,5.041199,2.423221,20944.451311,69.656236,36.781835,85.264824
1047987,torchscript_stablecode-3b-completion,9,107.748,5.041199,2.423221,20944.451311,69.656236,36.781835,85.264824
1047988,torchscript_stablecode-3b-completion,9,107.748,5.041199,2.423221,20944.451311,69.656236,36.781835,85.264824
1047989,torchscript_stablecode-3b-completion,9,107.748,5.041199,2.423221,20944.451311,69.656236,36.781835,85.264824


In [314]:
df_ag_01.columns

Index(['label', 'experiment', 'config_duration', 'avg_utilization_gpu_config',
       'avg_utilization_memory_config', 'avg_memory_used_config',
       'avg_power_draw_config', 'avg_temperature_gpu_config',
       'avg_used_memory_pct_mib'],
      dtype='object')

In [315]:
#energibridge change
#df_ag_01_variables = ['avg_cpu_usage_config', 'avg_USED_MEMORY','avg_used_memory_pct_config'] 
#nvidia
df_ag_01_variables = ['config_duration', 'avg_utilization_gpu_config','avg_utilization_memory_config', 'avg_memory_used_config','avg_used_memory_pct_mib']

df_ag_02 = df_ag_01.groupby(['label','experiment'])[df_ag_01_variables].mean().reset_index()
df_ag_02


,label,experiment,config_duration,avg_utilization_gpu_config,avg_utilization_memory_config,avg_memory_used_config,avg_used_memory_pct_mib
0,onnx_bloomz-1b1,1,1046.176,17.931602,2.343146,2648.308129,10.781258
1,onnx_bloomz-1b1,2,1039.802,18.035811,2.330817,2625.757259,10.689453
2,onnx_bloomz-1b1,3,1157.241,16.228949,2.094990,3134.658490,12.761189
3,onnx_bloomz-1b1,4,1039.818,18.074712,2.333785,2712.607278,11.043019
4,onnx_bloomz-1b1,5,1038.632,18.064722,2.341730,2629.047573,10.702848
...,...,...,...,...,...,...,...
355,torchscript_tinyllama,6,107.464,2.165103,0.922139,13469.904315,54.835956
356,torchscript_tinyllama,7,107.188,2.160865,0.920978,13449.905927,54.754543
357,torchscript_tinyllama,8,107.658,2.134831,0.904494,13469.906367,54.835965
358,torchscript_tinyllama,9,107.137,2.174036,0.922860,13602.919097,55.377459


In [352]:
# time
#df_ag_02 = pd.read_csv(tables_dir + 'final_inference_time_data.csv',index_col=None)
# energy
df_ag_02 = pd.read_csv(tables_dir + 'final_energy_data.csv',index_col=None) #change

df_ag_02

,label,experiment,CPU_energy,GPU_energy,global_energy
0,idle_idle,1,5356.125122,0.000000,5356.125122
1,idle_idle,2,5350.155334,0.000000,5350.155334
2,idle_idle,3,5174.852402,0.000000,5174.852402
3,idle_idle,4,5180.039169,0.000000,5180.039169
4,idle_idle,5,5366.871078,0.000000,5366.871078
...,...,...,...,...,...
365,torchscript_tinyllama,6,2479.216080,7022.551625,9501.767705
366,torchscript_tinyllama,7,2528.263107,7052.688061,9580.951168
367,torchscript_tinyllama,8,2645.645905,6716.034659,9361.680564
368,torchscript_tinyllama,9,2569.570862,7063.159418,9632.730280


In [353]:
ep

'cudaep'

In [354]:
# change if energy
from_name = 'energy' # inference_time, energy, energibridge, nvidia

if from_name == 'energy' and ep == 'cpuep':
    df_ag_02['GPU_energy'] = 0
    df_ag_02['global_energy'] = df_ag_02['CPU_energy']
    df_ag_02

In [355]:
df_ag_02

,label,experiment,CPU_energy,GPU_energy,global_energy
0,idle_idle,1,5356.125122,0.000000,5356.125122
1,idle_idle,2,5350.155334,0.000000,5350.155334
2,idle_idle,3,5174.852402,0.000000,5174.852402
3,idle_idle,4,5180.039169,0.000000,5180.039169
4,idle_idle,5,5366.871078,0.000000,5366.871078
...,...,...,...,...,...
365,torchscript_tinyllama,6,2479.216080,7022.551625,9501.767705
366,torchscript_tinyllama,7,2528.263107,7052.688061,9580.951168
367,torchscript_tinyllama,8,2645.645905,6716.034659,9361.680564
368,torchscript_tinyllama,9,2569.570862,7063.159418,9632.730280


In [356]:
df_ag_02[['runtime', 'model']] = df_ag_02['label'].str.split('_', n=1, expand=True)
df_ag_02

,label,experiment,CPU_energy,GPU_energy,global_energy,runtime,model
0,idle_idle,1,5356.125122,0.000000,5356.125122,idle,idle
1,idle_idle,2,5350.155334,0.000000,5350.155334,idle,idle
2,idle_idle,3,5174.852402,0.000000,5174.852402,idle,idle
3,idle_idle,4,5180.039169,0.000000,5180.039169,idle,idle
4,idle_idle,5,5366.871078,0.000000,5366.871078,idle,idle
...,...,...,...,...,...,...,...
365,torchscript_tinyllama,6,2479.216080,7022.551625,9501.767705,torchscript,tinyllama
366,torchscript_tinyllama,7,2528.263107,7052.688061,9580.951168,torchscript,tinyllama
367,torchscript_tinyllama,8,2645.645905,6716.034659,9361.680564,torchscript,tinyllama
368,torchscript_tinyllama,9,2569.570862,7063.159418,9632.730280,torchscript,tinyllama


In [357]:
# Map the Size Range from model_df to df_to_save
df_ag_02 = df_ag_02.merge(model_df[['Model', 'size_group']], left_on='model', right_on='Model', how='left')

# Drop the redundant 'Model' column after merging
df_ag_02 = df_ag_02.drop(columns=['Model'])
df_ag_02

,label,experiment,CPU_energy,GPU_energy,global_energy,runtime,model,size_group
0,idle_idle,1,5356.125122,0.000000,5356.125122,idle,idle,NaN
1,idle_idle,2,5350.155334,0.000000,5350.155334,idle,idle,NaN
2,idle_idle,3,5174.852402,0.000000,5174.852402,idle,idle,NaN
3,idle_idle,4,5180.039169,0.000000,5180.039169,idle,idle,NaN
4,idle_idle,5,5366.871078,0.000000,5366.871078,idle,idle,NaN
...,...,...,...,...,...,...,...,...
365,torchscript_tinyllama,6,2479.216080,7022.551625,9501.767705,torchscript,tinyllama,2.0
366,torchscript_tinyllama,7,2528.263107,7052.688061,9580.951168,torchscript,tinyllama,2.0
367,torchscript_tinyllama,8,2645.645905,6716.034659,9361.680564,torchscript,tinyllama,2.0
368,torchscript_tinyllama,9,2569.570862,7063.159418,9632.730280,torchscript,tinyllama,2.0


In [358]:
#change
df_variables = ['CPU_energy', 	'GPU_energy', 	'global_energy'] #change
#df_variables = ['duration'] #change
#df_variables = df_ag_01_variables

In [359]:
df_ag_03 = df_ag_02.groupby(['label'])[df_variables].median().reset_index()
df_ag_03

,label,CPU_energy,GPU_energy,global_energy
0,idle_idle,5331.372978,0.000000,5331.372978
1,onnx_bloomz-1b1,51748.184044,62642.135541,114327.170499
2,onnx_bloomz-560m,10006.586121,18373.828223,28371.988127
3,onnx_codegemma-2b,35610.280609,77668.948789,113279.229398
4,onnx_codeparrot-small,9081.845177,13986.934956,23077.985648
5,onnx_phi2,40885.409996,87022.120454,127932.914739
6,onnx_pythia-410m,21988.519135,33065.796229,55228.025927
7,onnx_pythia1-4b,26553.710274,59124.014519,85651.403900
8,onnx_stablecode-3b,32293.401276,54655.184992,86823.784125
9,onnx_stablecode-3b-completion,47259.743469,106721.017104,154000.752026


In [360]:
# Split the 'label' column into 'runtime' and 'model'
df_ag_03[['runtime', 'model']] = df_ag_03['label'].str.split('_', n=1, expand=True)
df_ag_03

,label,CPU_energy,GPU_energy,global_energy,runtime,model
0,idle_idle,5331.372978,0.000000,5331.372978,idle,idle
1,onnx_bloomz-1b1,51748.184044,62642.135541,114327.170499,onnx,bloomz-1b1
2,onnx_bloomz-560m,10006.586121,18373.828223,28371.988127,onnx,bloomz-560m
3,onnx_codegemma-2b,35610.280609,77668.948789,113279.229398,onnx,codegemma-2b
4,onnx_codeparrot-small,9081.845177,13986.934956,23077.985648,onnx,codeparrot-small
5,onnx_phi2,40885.409996,87022.120454,127932.914739,onnx,phi2
6,onnx_pythia-410m,21988.519135,33065.796229,55228.025927,onnx,pythia-410m
7,onnx_pythia1-4b,26553.710274,59124.014519,85651.403900,onnx,pythia1-4b
8,onnx_stablecode-3b,32293.401276,54655.184992,86823.784125,onnx,stablecode-3b
9,onnx_stablecode-3b-completion,47259.743469,106721.017104,154000.752026,onnx,stablecode-3b-completion


In [361]:
df_ag_03['runtime'].unique()

array(['idle', 'onnx', 'torch', 'torchscript'], dtype=object)

In [362]:
# Map the Size Range from model_df to df_to_save
df_ag_03 = df_ag_03.merge(model_df[['Model', 'size_group']], left_on='model', right_on='Model', how='left')

# Drop the redundant 'Model' column after merging
df_ag_03 = df_ag_03.drop(columns=['Model'])
df_ag_03

,label,CPU_energy,GPU_energy,global_energy,runtime,model,size_group
0,idle_idle,5331.372978,0.000000,5331.372978,idle,idle,NaN
1,onnx_bloomz-1b1,51748.184044,62642.135541,114327.170499,onnx,bloomz-1b1,2.0
2,onnx_bloomz-560m,10006.586121,18373.828223,28371.988127,onnx,bloomz-560m,1.0
3,onnx_codegemma-2b,35610.280609,77668.948789,113279.229398,onnx,codegemma-2b,3.0
4,onnx_codeparrot-small,9081.845177,13986.934956,23077.985648,onnx,codeparrot-small,1.0
5,onnx_phi2,40885.409996,87022.120454,127932.914739,onnx,phi2,3.0
6,onnx_pythia-410m,21988.519135,33065.796229,55228.025927,onnx,pythia-410m,1.0
7,onnx_pythia1-4b,26553.710274,59124.014519,85651.403900,onnx,pythia1-4b,2.0
8,onnx_stablecode-3b,32293.401276,54655.184992,86823.784125,onnx,stablecode-3b,3.0
9,onnx_stablecode-3b-completion,47259.743469,106721.017104,154000.752026,onnx,stablecode-3b-completion,3.0


In [363]:

df_ag_04 = df_ag_03.groupby(['size_group','runtime'])[df_variables].mean().reset_index()
df_ag_04

,size_group,runtime,CPU_energy,GPU_energy,global_energy
0,1.0,onnx,13404.990673,20692.811506,34138.272887
1,1.0,torch,5193.326574,11250.868661,16432.524734
2,1.0,torchscript,1876.043694,5938.361817,7807.141250
3,2.0,onnx,32113.950743,55652.207433,87788.334537
4,2.0,torch,7287.234735,24247.469324,31525.715135
5,2.0,torchscript,2155.506113,6622.720055,8760.154779
6,3.0,onnx,39012.208838,81516.817835,120509.170072
7,3.0,torch,12196.071341,50052.112930,62301.620314
8,3.0,torchscript,2495.156120,7714.828673,10209.124732


In [364]:
save_dir

'D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/aggregated/'

In [365]:
from_name

'energy'

In [366]:
df_ag_02.columns

Index(['label', 'experiment', 'CPU_energy', 'GPU_energy', 'global_energy',
       'runtime', 'model', 'size_group'],
      dtype='object')

In [367]:
agg_level = 2 # change

if agg_level == 1:
    df_datasets = [ df_ag_01,df_ag_02, df_ag_03, df_ag_04,] #change
elif agg_level == 2:
    df_datasets = [ df_ag_02, df_ag_03, df_ag_04,] #change

SAVE_TABLES = True # change


#from_name = 'energibridge' #change inference_time, energy
for dataset in df_datasets:
    df_name = f"{from_name}_vars_{agg_level}.csv"
    dataset.to_csv(save_dir+df_name, index=False)  if SAVE_TABLES else print(f"SAVE_TABLES:{SAVE_TABLES}")   
    print(f"Saving in: {save_dir+df_name}")
    agg_level+=1

Saving in: D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/aggregated/energy_vars_2.csv
Saving in: D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/aggregated/energy_vars_3.csv
Saving in: D:/GAISSA/energy-repo/repo_sept_last_update/cudaep_nov_results/aggregated/energy_vars_4.csv
